<a href="https://colab.research.google.com/github/aleks-haksly/Simulative/blob/main/SQL/%D0%9A%D0%B0%D1%80%D1%8C%D0%B5%D1%80%D0%BD%D1%8B%D0%B9_%D0%B8%D0%BD%D1%82%D0%B5%D0%BD%D1%81%D0%B8%D0%B2%20/%D0%9A%D0%B0%D1%80%D1%8C%D0%B5%D1%80%D0%BD%D1%8B%D0%B9_%D0%B8%D0%BD%D1%82%D0%B5%D0%BD%D1%81%D0%B8%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sqlalchemy import text
from sqlalchemy import create_engine
import numpy as np
from google.colab import userdata
SQLALCHEMY_SILENCE_UBER_WARNING=1

In [2]:
engine = create_engine(userdata.get('elefantSQL'))

In [3]:
def select(sql):
    sql = text(sql)
    return pd.read_sql(sql, engine)

### Задача 1: ###
Необходимо составить отчет о клиентах, чьи успешные транзакции превышают сумму в 100 тысяч рублей. В отчете должны содержаться идентификаторы клиентов (customer_id) и суммы их успешных транзакций в российских рублях (amount_rur). Успешной транзакцией считается та, где значение флага success_flg равно TRUE. Отсортировать результат по идентификатору клиента (customer_id).

In [ ]:
sql = '''
SELECT
  customer_id,
  amount_rur
FROM
  transaction_tin
WHERE
  success_flg = 'true'
  AND customer_id in (
    SELECT
      DISTINCT customer_id
    FROM
      transaction_tin
    WHERE
      amount_rur > 100000
  )
ORDER BY
  customer_id
'''

In [ ]:
select(sql)

,customer_id,amount_rur
0,1,158200.00
1,1,500.00
2,1,25.00
3,3,2300.00
4,3,273155.81
5,4,176.24
6,4,308926.00


### Задача 2: ###
Вывести дату первой транзакции для клиентов, пришедших в 2023 году.

In [ ]:
sql = """
SELECT
  customer_id,
  min(transaction_dttm)
FROM
  transaction_tin
WHERE
  success_flg = true
  and customer_id in (
    SELECT
      customer_id
    FROM
      customer_tin
    WHERE
      date_part('year', start_dttm) = 2023
  )
GROUP BY
  customer_id
"""

In [ ]:
select(sql)

,customer_id,min
0,4,2023-01-09 16:26:00
1,5,2023-01-09 10:15:00


### Задача 3: ###
Вывести все транзакции, которые больше предыдущей и последующей транзакции по этому клиенту.

In [4]:
sql = """
WITH RankedTransactions AS (
  SELECT
    transaction_id,
    customer_id,
    amount_rur,
    transaction_dttm,
    LAG(amount_rur) OVER (PARTITION BY customer_id ORDER BY transaction_dttm) AS prev_amount_rur,
    LEAD(amount_rur) OVER (PARTITION BY customer_id ORDER BY transaction_dttm) AS next_amount_rur
  FROM
    transaction_tin
WHERE success_flg = true
)
SELECT
  transaction_id,
  customer_id
FROM
  RankedTransactions
WHERE
  amount_rur > prev_amount_rur AND
  amount_rur > next_amount_rur
"""

In [5]:
select(sql)

,transaction_id,customer_id
0,108,2
1,112,5


### Задача 4 ###
Для каждой транзакции посчитать количество дней, которые прошли успешно с момента первой транзакции по этому клиенту. Проставить флаг, относится ли транзакция к клиенту, начавшему обслуживаться в банке в 2023 году.

In [10]:
sql = """
SELECT
  transaction_id,
  customer_id,
  amount_rur,
  transaction_dttm,
  (transaction_dttm :: date - min(transaction_dttm :: date) OVER (
    PARTITION BY customer_id
    ORDER BY
      transaction_dttm
  ))::INTEGER AS days_count,
  CASE
    WHEN customer_id in (
      SELECT
        customer_id
      FROM
        customer_tin
      WHERE
        date_part('year', start_dttm) = 2023
    ) THEN TRUE
    ELSE FALSE
  END AS new_cust_flg
FROM
  transaction_tin
WHERE
  success_flg = True
"""

In [11]:
select(sql)

,transaction_id,customer_id,amount_rur,transaction_dttm,days_count,new_cust_flg
0,101,1,500.00,2023-01-05 10:12:00,0,False
1,102,1,25.00,2023-01-05 14:50:00,0,False
2,114,1,158200.00,2023-01-10 21:18:00,5,False
3,104,2,326.00,2023-01-08 08:44:00,0,False
4,108,2,12008.58,2023-01-09 11:20:00,1,False
5,117,2,200.00,2023-01-12 16:51:00,4,False
6,105,3,273155.81,2023-01-08 09:08:00,0,False
7,113,3,2300.00,2023-01-10 20:15:00,2,False
8,110,4,176.24,2023-01-09 16:26:00,0,True
9,111,4,308926.00,2023-01-09 18:32:00,0,True


### Задача 5 ###
Вывести имена клиентов, у которых есть как минимум один активный счет, открытый больше года назад (ранее 1 июня 2022 года). И которые за последние полгода (с 1 июня 2022 года по 1 июня 2023 года) по всем своим счетам имеют движение средств меньше, чем 7000 рублей.

In [14]:
sql = """
SELECT
  c.name,
  sum(t.amount) as sun_amount
FROM
  client_tin c
  LEFT JOIN account_tin a ON a.client_id = c.id
  LEFT JOIN transactions_tin2 t ON t.account_id = a.id
WHERE
  a.open_dt < '2022-06-01'
  AND a.close_dt is null
GROUP BY
  c.id,
  c.name
HAVING
  sum(t.amount) < 7000
"""

In [15]:
select(sql)

,name,sun_amount
0,Петров,6094.42
